In [78]:
import pandas as pd
import os
import urllib
import gzip

# Weather Data Preprocessor

Adopted from `GHCN_data_preprocessing.ipynb` at https://drive.google.com/file/d/1-ROib2DJIDWg1njvard4sJS8vGc7GrVT/view?usp=share_link.

Although `GHCN_data_preprocessing.ipynb` was helpful, using `readlines()` was very slow on such a large CSV file. When running locally, I found it would take around 3 days to parse through a single year of data. Therefore, since my computer was capable (32GB of RAM), I decided to load in the entire CSV into a pandas dataframe and perform the same processing steps.

# Download the data

If `wget` is not installed on your machine, you may want to try `curl URL > file.txt` or the `urllib` package in python.

In [88]:
%%cmd
mkdir data_ghcn
cd data_ghcn
curl https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt > ghcnd-stations.txt
curl https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2010.csv.gz > 2010.csv.gz
    

Microsoft Windows [Version 10.0.22621.819]
(c) Microsoft Corporation. All rights reserved.

C:\Data\berkeley_classes\fa22\data200-grad-project>mkdir data_ghcn


A subdirectory or file data_ghcn already exists.



C:\Data\berkeley_classes\fa22\data200-grad-project>cd data_ghcn

C:\Data\berkeley_classes\fa22\data200-grad-project\data_ghcn>curl https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt > ghcnd-stations.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.1M  100 10.1M    0     0  3127k      0  0:00:03  0:00:03 --:--:-- 3128k



C:\Data\berkeley_classes\fa22\data200-grad-project\data_ghcn>curl https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2010.csv.gz > 2010.csv.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  171M  100  171M    0     0  7223k      0  0:00:24  0:00:24 --:--:-- 20.4M



C:\Data\berkeley_classes\fa22\data200-grad-project\data_ghcn>    
C:\Data\berkeley_classes\fa22\data200-grad-project\data_ghcn>

In [89]:
# note, must manually download the years from above by
# changing the associated code (specifically, the years)
years_to_process = ['2010', '2011', '2012', '2013', '2014', '2015', '2016']

In [90]:
# unzip the data
import gzip
import shutil

for curr_year in years_to_process:
    with gzip.open('./data_ghcn/{}.csv.gz'.format(curr_year), 'rb') as f_in:
        with open('./data_ghcn/{}.csv'.format(curr_year), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [91]:
# get the station dataframe
def get_stations(filename='data_ghcn/ghcnd-stations.txt'):
    df = pd.read_csv(filename, '/t', header=None)
    df = df[0].str.split(expand=True)[[0, 1, 2, 3]]
    df.columns = ['Station', 'Latitude', 'Longitude', 'Elevation']
    return df

stations = get_stations()

C:\Users\Abatics\AppData\Local\Temp\ipykernel_14804\2670145683.py:8: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  stations = get_stations()
C:\Users\Abatics\AppData\Local\Temp\ipykernel_14804\2670145683.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename, '/t', header=None)


In [92]:
for curr_year in years_to_process:
    # read in the weather data csv file
    df = pd.read_csv("./data_ghcn/{}.csv".format(curr_year), header=None)\
                     .drop(columns=[4, 5, 6, 7])
    print('Finished reading ./data_ghcn/{}.csv'.format(curr_year))

    # rename the columns
    df.columns = ['Station', 'Date', 'Element', 'Value']

    # get only rows with TAVG and PRCP
    prcp_tavg = df[(df['Element']=='TAVG') | (df['Element']=='PRCP')]

    # drop rows that do not have associated station names
    prcp_tavg_valid = prcp_tavg[prcp_tavg['Station']\
                                .isin(stations['Station'])]

    # pivot_table to get TAVG and PRCP as columns
    prcp_tavg_pivoted = prcp_tavg_valid.pivot_table(index=['Date', 'Station'],
                                values='Value',
                                columns='Element')\
                                .dropna()\
                                .rename_axis(None, axis=1)\
                                .reset_index()
    print('Finished Pivot Table for {}'.format(curr_year))

    # reorder columns
    prcp_tavg_pivoted = prcp_tavg_pivoted[['Station', 'Date', 'TAVG', 'PRCP']]

    # merge PRCP and TAVG table with station dataframe
    df_merged = prcp_tavg_pivoted.merge(stations,
                                        left_on='Station',
                                        right_on='Station',
                                        how='left')

    # change Date column to datetime type
    df_merged['Date'] = pd.to_datetime(df_merged['Date'], format='%Y%m%d')

    # change other numerical columns to float
    for c in ['Latitude', 'Longitude', 'TAVG', 'PRCP', 'Elevation']:
        df_merged[c] = df_merged[c].astype(float)
        
    print('Finished merge with station and type conversion for {}'\
          .format(curr_year))

#     display(df_merged)

    # save to CSV
    print('Started save for {}'.format(curr_year))
    df_merged.to_csv('data_ghcn/daily_global_weather_{}.csv'.format(curr_year))
    print('Finished saving CSV to ./data_ghcn/daily_global_weather_{}.csv'\
          .format(curr_year))
    print('Finished processing for {}'.format(curr_year))
    print()

Finished reading ./data_ghcn/2010.csv
Finished Pivot Table for 2010
Finished merge with station and type conversion for 2010
Started save for 2010
Finished saving CSV to ./data_ghcn/daily_global_weather_2010.csv
Finished processing for 2010

Finished reading ./data_ghcn/2011.csv
Finished Pivot Table for 2011
Finished merge with station and type conversion for 2011
Started save for 2011
Finished saving CSV to ./data_ghcn/daily_global_weather_2011.csv
Finished processing for 2011

Finished reading ./data_ghcn/2012.csv
Finished Pivot Table for 2012
Finished merge with station and type conversion for 2012
Started save for 2012
Finished saving CSV to ./data_ghcn/daily_global_weather_2012.csv
Finished processing for 2012

Finished reading ./data_ghcn/2013.csv
Finished Pivot Table for 2013
Finished merge with station and type conversion for 2013
Started save for 2013
Finished saving CSV to ./data_ghcn/daily_global_weather_2013.csv
Finished processing for 2013

Finished reading ./data_ghcn/201